## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 50.0 MB/s 
     |████████████████████████████████| 163 kB 83.3 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.6 MB/s 
     |████████████████████████████████| 212 kB 83.6 MB/s 
     |████████████████████████████████| 115 kB 68.5 MB/s 
     |████████████████████████████████| 127 kB 78.9 MB/s 
     |████████████████████████████████| 115 kB 81.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 76.3 MB/s 
     |████████████████████████████████| 166 kB 74.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 80.6 MB/s 
     |████████████████████████████████| 162 kB 80.1 MB/s 
     |████████████████████████████████| 162 kB 79.3 MB/s 
     |████████████████████████████████| 158 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 84.1 MB/s 
     |████████████████████████████████| 157 kB 81.0 MB/s 
     |████████████████████████████████| 157 kB 83.4 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 67.0 MB/s 
     |████████████████████████████████| 157 kB 79.8 MB/s 
     |████████████████████████████████| 157 kB 77.5 MB/s 
     |████████████████████████████████| 156 kB 83.8 MB/s 


In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

 76% 29.0M/38.1M [00:00<00:00, 158MB/s] 
100% 38.1M/38.1M [00:00<00:00, 172MB/s]


In [5]:
!unzip /content/aihub-groom.zip

Archive:  /content/aihub-groom.zip
  inflating: data/Training/talksets-train-1.json  
  inflating: data/Training/talksets-train-2.json  
  inflating: data/Training/talksets-train-3.json  
  inflating: data/Training/talksets-train-4.json  
  inflating: data/Training/talksets-train-5.json  
  inflating: data/Validation/talksets-train-6.json  


In [30]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    DefaultDataCollator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [7]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## readjson

In [78]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    ids = []
    for group in squad_dict:
        for passage in group['sentences']:
            id = passage['id']
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 1:
                continue
            if intensity < 1:
                texts.append(text)
                ids.append(id)

    return texts, ids

In [79]:
import pandas as pd

texts = []
ids = []
for i in range(1,6):
    file_path =  f'/content/data/Training/talksets-train-{i}.json'
    ttexts, iids = read_aihub(file_path)
    texts += ttexts
    ids += iids
moral_df = pd.DataFrame(list(zip(ids, texts)),
            columns =['ids', 'texts'])

<_io.BufferedReader name='/content/data/Training/talksets-train-1.json'>
<_io.BufferedReader name='/content/data/Training/talksets-train-2.json'>
<_io.BufferedReader name='/content/data/Training/talksets-train-3.json'>
<_io.BufferedReader name='/content/data/Training/talksets-train-4.json'>
<_io.BufferedReader name='/content/data/Training/talksets-train-5.json'>


In [80]:
moral_df

,ids,texts
0,615d33ea78290791e9561d5c-3,쉴드가 아니라 국가가 면제해준거야.
1,615d33fc78290791e9566c56-2,짐승입장도 들어봐야지ㅋㅋ
2,61921b89b1bee6a4bd7bdd1f-1,[UNK]이 이제 급하긴 한가보네
3,61921b89b1bee6a4bd7bdd1f-2,깐부라고 부르기까지 하다니
4,615d33f378290791e95644c2-1,어머니 간 이식 해 주기 싫어서 도망쳤다고...?
...,...,...
162943,616922089cbb21a3042d9bcd-3,와 넌 모욕죄 낼 통장 잔고가 넉넉한가봐?
162944,6152b6b7aab10bca18b8ff9f-1,바나나 먹으면 나한테 반하나?
162945,61651454ebf4829a1b39f676-3,오죽 하면 그러겠나
162946,615fb4a1dcbfe5fa32e5ce63-2,신경 끄세요.


In [81]:
from datasets import Dataset

moral_data = Dataset.from_pandas(moral_df)

In [82]:
moral_data

Dataset({
    features: ['ids', 'texts'],
    num_rows: 162948
})

## tokenize

In [83]:
model_name = 'beomi/KcELECTRA-base-v2022'
model_version = 'groom2team/pj3_class_kcelectra/pytorch_finetuned:v8'
max_length = 128

In [84]:
import wandb
run = wandb.init()
artifact1 = run.use_artifact(model_version, type='model')
artifact_dir1 = artifact1.download()

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(artifact_dir1+'/best_model_at_end/pytorch_model.bin'))

tokenizer = AutoTokenizer.from_pretrained(model_name)

wandb: Downloading large artifact pytorch_finetuned:v8, 489.19MB. 7 files... 
wandb:   7 of 7 files downloaded.  
Done. 0:0:0.1
Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were

In [85]:
def tokenizeWithoutLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    return tokenized_datas

In [86]:
device = torch.device('cuda')
model = model.to(device)

In [87]:
def get_softmax(text):
    texts = []
    texts.append({'ids': 0, 'texts' : text})
    tokens = tokenizer.encode(text)
    for i in range(1, len(tokens) - 1):
        texts.append({'ids': i, 'texts' : tokenizer.decode(tokens[1:i] + tokens[i+1:len(tokens)-1])})
    line_data = Dataset.from_list(texts)
    print(line_data)
    line_tokenized_datasets = line_data.map(tokenizeWithoutLabel, batched=True, remove_columns=line_data.column_names)
    data_collator = DefaultDataCollator(return_tensors="pt")
    line_loader = torch.utils.data.DataLoader(line_tokenized_datasets, batch_size=len(line_tokenized_datasets),
                                            shuffle=False, collate_fn=data_collator,
                                            num_workers=0)
    for id_inputs in line_loader:
        torch.cuda.empty_cache()
        inputs = {}
        inputs['input_ids'] = id_inputs['input_ids'].to(device)
        inputs['token_type_ids'] = id_inputs['token_type_ids'].to(device)
        inputs['attention_mask'] = id_inputs['attention_mask'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        softmax = logits.softmax(dim=-1)
        for i, prob in enumerate(softmax):
            print(prob, texts[i])
        print(text)


In [89]:
for line in moral_data:
    get_softmax(line['texts'])

Dataset({
    features: ['ids', 'texts'],
    num_rows: 9
})


  0%|          | 0/1 [00:00<?, ?ba/s]

tensor([0.9700, 0.0300], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 0, 'texts': '쉴드가 아니라 국가가 면제해준거야.'}
tensor([0.9744, 0.0256], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 1, 'texts': '##가 아니라 국가가 면제해준거야.'}
tensor([0.9792, 0.0208], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 2, 'texts': '쉴드 아니라 국가가 면제해준거야.'}
tensor([0.8221, 0.1779], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 3, 'texts': '쉴드가 국가가 면제해준거야.'}
tensor([0.9771, 0.0229], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 4, 'texts': '쉴드가 아니라 면제해준거야.'}
tensor([0.9478, 0.0522], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 5, 'texts': '쉴드가 아니라 국가가해준거야.'}
tensor([0.9038, 0.0962], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 6, 'texts': '쉴드가 아니라 국가가 면제거야.'}
tensor([0.9274, 0.0726], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 7, 'texts': '쉴드가 아니라 국가가 면제해준.'}
tensor([0.9593, 0.0407], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 8, 'texts': '쉴드가 아니라 국가가 면제해준거야'}
쉴드가 아니라 국

  0%|          | 0/1 [00:00<?, ?ba/s]

tensor([0.7589, 0.2411], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 0, 'texts': '짐승입장도 들어봐야지ㅋㅋ'}
tensor([0.9571, 0.0429], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 1, 'texts': '##입장도 들어봐야지ㅋㅋ'}
tensor([0.5068, 0.4932], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 2, 'texts': '짐승도 들어봐야지ㅋㅋ'}
tensor([0.7562, 0.2438], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 3, 'texts': '짐승입장 들어봐야지ㅋㅋ'}
tensor([0.7847, 0.2153], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 4, 'texts': '짐승입장도봐야지ㅋㅋ'}
tensor([0.7014, 0.2986], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 5, 'texts': '짐승입장도 들어ㅋㅋ'}
tensor([0.9499, 0.0501], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 6, 'texts': '짐승입장도 들어봐야지'}
짐승입장도 들어봐야지ㅋㅋ
Dataset({
    features: ['ids', 'texts'],
    num_rows: 7
})


...

  0%|          | 0/1 [00:00<?, ?ba/s]

tensor([0.9959, 0.0041], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 0, 'texts': '은근 젊은 층 중에 [UNK] 지지자들 많음.'}
tensor([0.9944, 0.0056], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 1, 'texts': '젊은 층 중에 [UNK] 지지자들 많음.'}
tensor([0.9936, 0.0064], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 2, 'texts': '은근 층 중에 [UNK] 지지자들 많음.'}
tensor([0.9955, 0.0045], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 3, 'texts': '은근 젊은 중에 [UNK] 지지자들 많음.'}
tensor([0.9964, 0.0036], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 4, 'texts': '은근 젊은 층 [UNK] 지지자들 많음.'}
tensor([0.9967, 0.0033], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 5, 'texts': '은근 젊은 층 중에 지지자들 많음.'}
tensor([0.9842, 0.0158], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 6, 'texts': '은근 젊은 층 중에 [UNK] 많음.'}
tensor([0.9919, 0.0081], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 7, 'texts': '은근 젊은 층 중에 [UNK] 지지자들.'}
tensor([0.9959, 0.0041], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 8, '

  0%|          | 0/1 [00:00<?, ?ba/s]

tensor([0.9944, 0.0056], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 0, 'texts': '맞아. 카리스마 있는 리더처럼 느껴져.'}
tensor([0.9947, 0.0053], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 1, 'texts': '. 카리스마 있는 리더처럼 느껴져.'}
tensor([0.9947, 0.0053], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 2, 'texts': '맞아 카리스마 있는 리더처럼 느껴져.'}
tensor([0.9897, 0.0103], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 3, 'texts': '맞아. 있는 리더처럼 느껴져.'}
tensor([0.9933, 0.0067], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 4, 'texts': '맞아. 카리스마 리더처럼 느껴져.'}
tensor([0.9933, 0.0067], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 5, 'texts': '맞아. 카리스마 있는처럼 느껴져.'}
tensor([0.9943, 0.0057], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 6, 'texts': '맞아. 카리스마 있는 리더 느껴져.'}
tensor([0.9877, 0.0123], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 7, 'texts': '맞아. 카리스마 있는 리더처럼.'}
tensor([0.9937, 0.0063], device='cuda:0', grad_fn=<UnbindBackward0>) {'ids': 8, 'texts': '맞아. 카리스마 있는 리더처럼 느

KeyboardInterrupt: ignored

In [ ]:
train_tokenized_datasets

## make train dataset

## train with sweep

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'eval/loss'},
    'parameters': 
    {
        'batch_size': {'values': [256]},
        'epochs': {'values': [5]},
        'lr': {'values': [1e-5]},
        'scheduler': {'values': ['linear', 'cosine', 'constant']}
     }
}

In [ ]:
max_batch_size = 256
def train():
    torch.cuda.empty_cache()
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    grouped_params = model.parameters()
    run = wandb.init(config=sweep_configuration, entity="groom2team")
    batch_size = wandb.config.batch_size if wandb.config.batch_size < max_batch_size else max_batch_size
    gradient_accumulation_steps= wandb.config.batch_size // max_batch_size
    epochs = wandb.config.epochs
    total_steps = int(len(train_tokenized_datasets)/wandb.config.batch_size*epochs)
    learning_rate = wandb.config.lr
    data_collator = default_data_collator
    grouped_params = model.parameters()
    optimizer=AdamW(grouped_params, lr=learning_rate)
    scheduler_type = wandb.config.scheduler
    if scheduler_type == 'linear':
        scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=0,
                                                    num_training_steps=total_steps+1)
    elif scheduler_type == 'cosine':
        scheduler=get_cosine_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps+1)
    elif scheduler_type == 'constant':
        scheduler=get_constant_schedule(optimizer=optimizer)
    optimizers = optimizer, scheduler
    args = TrainingArguments(
        f"{model_name}-finetuned",
        evaluation_strategy = "steps",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        report_to="wandb",
        run_name="utopia",
        logging_steps = total_steps//200,
        eval_steps = total_steps//100,
        save_steps = total_steps//100,
        weight_decay=0.0,
        save_total_limit = 2,
        load_best_model_at_end=True
    )
    trainer = Trainer(
        model,
        args,
        train_dataset=train_tokenized_datasets,
        eval_dataset=dev_tokenized_datasets,
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
        optimizers=optimizers
    )
    trainer.train()# train 하고
    trainer.save_model(output_dir= 'pytorch_finetuned') # trainer에서 실행된 model save
    artifact = wandb.Artifact(name='pytorch_finetuned', type='model') # wandb에 해당 모델 version 관리.
    artifact.add_dir('pytorch_finetuned', name='best_model_at_end')
    run.log_artifact(artifact)

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='pj3_class_kcelectra', entity='groom2team')
count = 3

Create sweep with ID: eacjzlg5
Sweep URL: https://wandb.ai/groom2team/pj3_class_kcelectra/sweeps/eacjzlg5


In [ ]:
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: lrcyhiso with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	lr: 5e-05
wandb: 	scheduler: linear


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'c

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 236856
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 4630
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
46,0.378600,0.337113
92,0.318300,0.292265
138,0.322700,0.289405
184,0.284000,0.290951
230,0.296800,0.290658
276,0.289500,0.282857
322,0.275000,0.280973
368,0.283000,0.275297
414,0.268900,0.278083
460,0.281600,0.275639


...

wandb: Agent Starting Run: y75qh4ca with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	lr: 5e-05
wandb: 	scheduler: constant
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--beomi--KcELECTRA-base-v2022/snapshots/4431b6c7ad00f82fd50880864574cef97e0a368b/config.json
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base-v2022",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
PyTorch: setting up devices
***** Running training *****
  Num examples = 236856
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 4630
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss
46,0.371800,0.336067
92,0.321900,0.302948
138,0.321300,0.289139
184,0.281700,0.288501
230,0.300600,0.291811
276,0.290600,0.283777
322,0.272100,0.285336
368,0.285700,0.274968
414,0.268600,0.281644
460,0.286400,0.277435


***** Running Evaluation *****
  Num examples = 59215
  Batch size = 256
Saving model checkpoint to beomi/KcELECTRA-base-v2022-finetuned/checkpoint-46
Configuration saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-46/config.json
Model weights saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-46/pytorch_model.bin
tokenizer config file saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-46/tokenizer_config.json
Special tokens file saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-46/special_tokens_map.json
Deleting older checkpoint [beomi/KcELECTRA-base-v2022-finetuned/checkpoint-828] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 59215
  Batch size = 256
Saving model checkpoint to beomi/KcELECTRA-base-v2022-finetuned/checkpoint-92
Configuration saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-92/config.json
Model weights saved in beomi/KcELECTRA-base-v2022-finetuned/checkpoint-92/pytorch_model.bin
tokenizer config file sav

eval/loss,█▅▃▃▃▃▃▂▂▂▂▁▂▂▂▂▁▁▁▃▃▄▃▃▃▃▃▃▂
eval/runtime,▆▅█▄▄▄▃▃▃▂▃▂▄▃▂█▂▃▃▂▁▂▂▄▂▂▃▄▄
eval/samples_per_second,▃▄▁▅▅▅▆▆▆▇▆▇▅▆▇▁▇▆▆▇█▇▇▅▇▇▆▅▅
eval/steps_per_second,▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁██▁▁▁▁█▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▃▃▃▂▃▃▃▂▃▂▂▃▂▂▂▃▃▂▂▂▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
wandb.finish() # wandb 종료